# Hyperparameter search for FCN

In [ ]:
# Allows imports from modules in the project directory
import os
import sys
from pathlib import Path

CURRENT_DIRECTORY = Path(os.path.abspath('')).resolve()
MODULE_DIRECTORY = CURRENT_DIRECTORY.parent
PROJECT_DIRECTORY = MODULE_DIRECTORY.parents[1]
sys.path.extend([str(MODULE_DIRECTORY)])

print(f'Python {sys.version} on {sys.platform}')

In [ ]:
import keras_tuner as kt
import numpy as np
import tensorflow as tf
from tensorflow import keras

import utilities

In [ ]:
# Load and split dataset
data, labels, fids, velocities, angles = utilities.load_dataset(PROJECT_DIRECTORY / 'data/data_adp.pkl')
train_idx, test_idx, train_data, test_data = utilities.split_dataset(fids, labels, data)

# Get the corresponding labels
train_labels = labels[train_idx]
test_labels = labels[test_idx]

# Normalize dataset
utilities.normalize_data(train_data)
utilities.normalize_data(test_data)

In [ ]:
# Data generator allows to transform each batch in training and evaluate the test set after each epoche
class DataGenerator(utilities.BaseDataGenerator):
    # Pad each batch to the maximum length
    def transform(self, X: list) -> np.ndarray:
        maxlength = 0
        for item in X:
            maxlength = max(maxlength, len(item))

        return utilities.pad_data(X, maxlength)

In [ ]:
# Initialize data generators
train_generator = DataGenerator(train_data, train_labels, batch_size=32)
test_generator = DataGenerator(test_data, test_labels, batch_size=32)

In [3]:
# Function for creating the different models
def create_model(hp):
    model = keras.Sequential(name='fcn_model')

    model.add(keras.layers.Input(shape=(None, 6)))

    for idx in range(3):
        model.add(
            keras.layers.Conv1D(
                filters=hp.Choice(f'layer_{idx}_filters', values=[8, 16, 32, 64, 128]),
                kernel_size=hp.Choice(f'layer_{idx}_kernel', values=[3, 4, 5, 6]),
                activation='relu',
                padding=hp.Choice(f'layer_{idx}_padding', values=['same', 'valid', 'causal'])
            )
        )
        model.add(keras.layers.BatchNormalization())
        model.add(
            keras.layers.MaxPooling1D(
                pool_size=hp.Choice(f'layer_{idx}_pool', values=[2, 3, 4, 5])
            )
        )

    model.add(keras.layers.GlobalAveragePooling1D())

    model.add(keras.layers.Dense(
        units=hp.Choice(f'dense_units', values=[8, 16, 32, 64]),
        activation='sigmoid')
    )
    model.add(keras.layers.Dense(4, activation='softmax'))

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [4]:
# Using Hyperband to efficiently get the best result in the search space
tuner = kt.Hyperband(
    create_model,
    objective='val_accuracy',
    max_epochs=10,
    factor=4,
    directory='hyper-search',
    project_name='my_fcn_search'
)

In [ ]:
# Stop early if there is no improvement on the validation loss
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# Start search (this may take a very long while)
tuner.search(train_generator, epochs=10, callbacks=[stop_early], validation_data=test_generator)

In [ ]:
# Print the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(best_hps.values)

In [ ]:
# The best model could be saved like this:
best_model = tuner.get_best_models()[0]
best_model.save(CURRENT_DIRECTORY / 'models/my_fcn_model_name')